In [4]:
#Import Libraries
import pandas as pd
import json
import numpy as np
import re

In [5]:
#Format meter data to be compatible with Pandas
with open('data/parkingMeters.json') as f:
    data = json.load(f)

json_df_format = {}

for meter in data['features']:
  attributes = meter.pop('attributes')
  geometry = meter.pop('geometry')
  attributes['LATITUDE'] = geometry['y']
  attributes['LONGITUDE'] = geometry['x']
  meter_id = attributes.pop('METER_ID')
  json_df_format[meter_id] = attributes

In [6]:
#Import formatted meter data
with open('data/formattedMeterData.json', 'w') as fp:
    json.dump(json_df_format, fp)
meters = pd.read_json('data/formattedMeterData.json', orient='index')

#Remove Taxi Zones
meters['TAXI_ZONE_HOURS'] = meters['TAXI_ZONE_HOURS'].apply(lambda x: str(x))
meters = meters[meters['TAXI_ZONE_HOURS'] == 'None']

#Remove Handicap Zones
meters = meters[meters['HANDICAP'] == 0]

#Text to Hour of Day
text_to_hod = {'3AM':3, '4PM':16, '3PM':15, '7AM':7, '6PM':18, '10AM':10, '9AM':9, '6AM':6, 'None':0, '0':0}
meters['TOW_AWAY_HOURS'] = meters['TOW_AWAY_HOURS'].apply(lambda x: str(x))
meters['TOW_AWAY_START'] = meters['TOW_AWAY_HOURS'].apply(lambda x: text_to_hod[x.split('-')[0].replace(" ","")])
meters['TOW_AWAY_END'] = meters['TOW_AWAY_HOURS'].apply(lambda x: text_to_hod[x.split('-')[-1].replace(" ","")])

#Only extract meter locations
meters = meters[['LATITUDE','LONGITUDE']]
meters.head()

,LATITUDE,LONGITUDE
WR158,39.964668,-83.004321
WR110,39.964754,-83.003614
WR36,39.964864,-83.002652
NG16,39.963145,-83.004304
ET462,39.967236,-82.992277


In [7]:
#Import RvBsm
RvBsm = pd.read_csv('data/RvBsm.csv')
RvBsm.head()

,Device,Trip,Time,RvDevice,NativeFlag,LocalTimeMS,BsmPsId,BasicVehClass,RvRandomId,Latitude,Longitude,Elevation,Heading,GpsSpeed,BrakeStatus,YawRate,LongAccel
0,2004,12,20450,2494,0,80282393,1,0,1368,40.283614,-83.529785,291.9,228.7125,25.50,0,-0.97,0.07
1,2004,12,20500,2494,0,80282893,1,0,1368,40.283537,-83.529898,292.0,227.9500,25.40,0,-1.46,0.01
2,2004,12,20550,2494,0,80283393,1,0,1368,40.283458,-83.530009,292.1,226.9500,25.30,0,-2.19,0.01
3,2004,12,20600,2494,0,80283893,1,0,1368,40.283377,-83.530117,292.1,225.5000,25.32,0,-2.92,0.07
4,2004,12,20650,2494,0,80284393,1,0,1368,40.283294,-83.530222,292.2,224.0875,25.32,0,-2.92,0.07


In [26]:
#Use to distance to destination
def coord_to_dist(df, dest_lat, dest_long):
    print(dest_lat)
    print(dest_long)
    long_dist = (df['Longitude'] - dest_long)*69
    lat_dist = (df['Latitude'] - dest_lat)*69
    return (long_dist**2 + lat_dist**2)**0.5

In [27]:
def label_parking_availability(df):
    """
    Due to the lack of labeling in our data, we will make our own inferences to label the data based on 
    psychological behaviors when driving.
    
    Parameters
    ----------
    df : Pandas Dataframe w/ time column in ms and features
        Each DataFrame should be a unique trip sorted from by time, descending
    T : int
        maximum length of the period for each data sample
    Returns
    -------
    cleaned_df : Pandas Dataframe w/ time column in ms and features
        Returns the dataframe with only the features that can be used to predict parking availability.
        The Dataframe should have df_time % T entries, with the first extraneous entries removed
    labels : 1D array-like list
        Returns labels for each row where the last T rows are "Available Parking", 
        the sets of T rows where the car is within walking_distance of the destination and going under max_speed are 
        "Looking for Parking", and the rest is "Not Looking for Parking"
    """
    #Period in milliseconds
    T = 180000 #3 minutes
    
    #Walking Distance in miles
    walking_distance = 0.25 #5 Blocks in miles
    
    #Speed in m/s
    max_speed = 7 #About 15mph, the speed limit in a parking lot
    
    end_time = max(df['LocalTimeMS'])
    
    
    #Available Parking
    ap_df = df[df['LocalTimeMS'] >= end_time - T]
    ap_label = np.array(['Available Parking' for i in range(len(lfp))])
    
    #Looking for Parking
    lfp_df = df[(df['LocalTimeMS'] < end_time - T)]
    lfp_start_time = lfp_df[(lfp_df['DistanceToDestination'] > walking_distance) | (lfp_df['GpsSpeed'] > max_speed)]['LocalTimeMS']
    lfp_df = lfp_df[lfp_df['LocalTimeMS'] > (lfp_start_time - ((end_time-lfp_start_time)%T))]
    
    #Not looking for parking
    nlfp_df = lfp_df[(lfp_df['LocalTimeMS'] < min(lfp_df['LocalTimeMS']))]
    nlfp_start_time = ((max(nlfp_df['LocalTimeMS']) - min(nlfp_df['LocalTimeMS'])) // T) * T
    nlfp_df = nlfp_df[]
    return df

In [28]:
RvBsm['Trip'].dtype

dtype('int64')

40.213693

In [34]:
device2004_trip12 = RvBsm[(RvBsm['Device'] == 2004) & (RvBsm['Trip'] == 12)].sort_values('LocalTimeMS',ascending=False)

dest_lat = device2004_trip12.iloc[0]['Latitude']
dest_long = device2004_trip12.iloc[0]['Longitude']

device2004_trip12['DistanceToDestination'] = coord_to_dist(device2004_trip12, dest_lat, dest_long)

device2004_trip12.head(20)
#label_parking_availability(device2004_trip12).head()

40.213693
-83.30840500000001


,Device,Trip,Time,RvDevice,NativeFlag,LocalTimeMS,BsmPsId,BasicVehClass,RvRandomId,Latitude,Longitude,Elevation,Heading,GpsSpeed,BrakeStatus,YawRate,LongAccel,DistanceToDestination
1502,2004,12,100150,2494,0,81079358,1,0,1368,40.213693,-83.308405,268.1,49.5875,10.880000,0,-7.08,2.55,0.000000
1501,2004,12,100100,2494,0,81078858,1,0,1368,40.213693,-83.308405,268.1,49.5875,10.880000,0,-7.08,2.55,0.000000
1500,2004,12,100050,2494,0,81078358,1,0,1368,40.213693,-83.308405,268.1,49.5875,10.880000,0,-7.08,2.55,0.000000
1499,2004,12,100000,2494,0,81077858,1,0,1368,40.213674,-83.308433,268.1,51.5625,10.440000,0,-8.54,2.41,0.002335
1498,2004,12,99950,2494,0,81077358,1,0,1368,40.213645,-83.308478,268.0,59.4625,9.139999,0,-15.13,1.74,0.006028
1497,2004,12,99900,2494,0,81076858,1,0,1368,40.213623,-83.308522,268.0,67.9375,8.260000,0,-24.65,1.31,0.009408
1496,2004,12,99850,2494,0,81076358,1,0,1368,40.213604,-83.308597,267.9,93.2625,6.560000,0,-23.68,3.03,0.014602
1495,2004,12,99800,2494,0,81075858,1,0,1368,40.213603,-83.308605,267.9,95.9375,5.920000,0,-20.99,2.22,0.015133
1494,2004,12,99750,2494,0,81075358,1,0,1368,40.213605,-83.308637,267.8,104.5000,4.520000,0,-19.04,3.42,0.017121
1493,2004,12,99700,2494,0,81074858,1,0,1368,40.213608,-83.308660,267.7,112.0625,3.200000,0,-8.78,3.50,0.018547


In [ ]:
api_key = 'API__KEY'